## Topic Clustering 

Source Materials: https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920

### Import required packages 

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Nick's
[nltk_data]     Laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

### Prepare stopwords 

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Load dataset

In [4]:
df = pd.DataFrame(pd.read_csv("modeling.csv"))
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Tech_Title,Lead_Inventor,Disclosure_Date,Division_Department,Owners,Alternate_Title,Tech_Publisher_Featured,Total_Budget,Ability_of_investigator_to_continue_research,...,Lawfirm,Attorney,Inventors_0,Inventors_1,Inventors_2,Inventors_3,Inventors_4,Number_of_Inventors,Application_Type,Licensed
0,0,Light Collectors in Cylinderical Geometry,Roland Winston,NaN,"Physical Sciences Division (PSD), Physics",University of Chicago,,No,0.0,No,...,"Marshall, Gerstein & Borun",NaN,Roland Winston,NaN,NaN,NaN,NaN,1.0,US Utility,Yes
1,1,Radiant Energy Collector,Roland Winston,NaN,"Physical Sciences Division (PSD), Physics",University of Chicago,,No,0.0,No,...,"Marshall, Gerstein & Borun","Borun, Michael",Roland Winston,NaN,NaN,NaN,NaN,1.0,US Utility,Yes
2,2,Cylindrical Radiant Energy Direction Device Wi...,Roland Winston,NaN,"Physical Sciences Division (PSD), Physics",University of Chicago,,No,0.0,No,...,"Marshall, Gerstein & Borun","Borun, Michael",Roland Winston,NaN,NaN,NaN,NaN,1.0,US Utility,Yes
3,3,Ideal Light Concentrators with Reflector Gaps,Roland Winston,NaN,"Physical Sciences Division (PSD), Physics",University of Chicago,,No,0.0,No,...,"Marshall, Gerstein & Borun",NaN,Roland Winston,NaN,NaN,NaN,NaN,1.0,US Utility,Yes
4,4,Nonimaging Radiant Energy Direction Device,Roland Winston,NaN,"Physical Sciences Division (PSD), Physics",University of Chicago,,No,0.0,No,...,"Marshall, Gerstein & Borun",NaN,Roland Winston,NaN,NaN,NaN,NaN,1.0,US Utility,Yes


###  Convert Titles to list and tokenize 

In [5]:
# Convert to list 
data = df.Tech_Title.values.tolist()

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['light', 'collectors', 'in', 'cylinderical', 'geometry']]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Create Bigram and Trigram models 

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['light', 'collectors', 'in', 'cylinderical', 'geometry']


### Remove Stopwords, make bigrams and lemmatize

In [7]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Call functions on Title data

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['light', 'collector', 'cylinderical', 'geometry']]


### Create data dictionary and corpus for topic modeling 

In [9]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Develop LDA Topic Model 

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### View LDA topics 

In [11]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.089*"image" + 0.038*"automate" + 0.035*"medical" + 0.028*"optical" + '
  '0.023*"lung" + 0.023*"identification" + 0.022*"computer" + '
  '0.019*"computerized" + 0.017*"analysis" + 0.016*"technique"'),
 (1,
  '0.058*"protein" + 0.044*"human" + 0.041*"relate" + 0.041*"nuclear" + '
  '0.035*"ubiquitous" + 0.030*"fragment" + 0.016*"vaccine" + 0.016*"radiation" '
  '+ 0.015*"process" + 0.015*"peptide"'),
 (2,
  '0.077*"gene" + 0.064*"receptor" + 0.040*"treatment" + 0.040*"herpe" + '
  '0.035*"virus" + 0.028*"function" + 0.026*"composition" + 0.025*"disease" + '
  '0.022*"induction" + 0.017*"base"'),
 (3,
  '0.060*"cell" + 0.055*"new" + 0.052*"detection" + 0.045*"plant" + '
  '0.042*"sequence" + 0.039*"identifying_centromere" + 0.026*"cancer" + '
  '0.023*"lesion" + 0.023*"improve" + 0.020*"promoter"'),
 (4,
  '0.167*"method" + 0.083*"system" + 0.052*"use" + '
  '0.041*"substituted_benzylguanine" + 0.020*"tumor" + 0.019*"modulation" + '
  '0.015*"expression" + 0.014*"generation" + 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Compute model Perplexity and Coherence score. We are looking for a low perplexity score and high coherence score

Perplexity:  -9.100127809875115

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -8.141246247042863


### Visualize the topic model 

In [13]:
# Visualize the topics
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
vis

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.204827  0.244709       1        1  24.734930
2     -0.017842 -0.333669       2        1  23.082947
3     -0.305578 -0.081014       3        1  20.125982
0     -0.175511  0.288616       4        1  16.887053
1      0.294105 -0.118643       5        1  15.169088, topic_info=                    Term         Freq        Total Category  logprob  loglift
28                method  1588.000000  1588.000000  Default  30.0000  30.0000
30                system   787.000000   787.000000  Default  29.0000  29.0000
39                 image   580.000000   580.000000  Default  28.0000  28.0000
131                 gene   681.000000   681.000000  Default  27.0000  27.0000
83              receptor   570.000000   570.000000  Default  26.0000  26.0000
...                  ...          ...          ...      ...      ...      ...
561      crystallization    31.545621    32.420349   Topic5  -5.2167   1.8586
1159            cysteine    52.515098    54.861196   Topic5  -4.7071   1.8422
3504          boric_acid    46.151458    50.671335   Topic5  -4.8363   1.7925
3506         lubrication    46.151458    50.671335   Topic5  -4.8363   1.7925
3568  determining_sensor    42.127991    74.588857   Topic5  -4.9275   1.3146

[203 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
625       1  0.988627      acid
921       3  0.993893  activate
1302      1  0.982666    active
630       1  0.988173  activity
50        3  0.994911     agent
...     ...       ...       ...
138       5  0.010877    vector
156       4  0.983989     viral
108       2  0.997863     virus
492       2  0.988407    viruse
1659      5  0.977983     wound

[243 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 4, 1, 2])